<a href="https://colab.research.google.com/github/dapopov-st/MiniLlamaSQL/blob/main/Fine_tune_CodeLlama_7b_Instruct_step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune premsql

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


/home/mainuser/anaconda3/envs/sqlft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-08 10:18:16,013] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/mainuser/anaconda3/envs/sqlft/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
print(torch.cuda.is_available())

True


In [3]:
import wandb # do from scipt later
wandb.login()
%env WANDB_PROJECT=sql-fine-tuning

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dpopovvelasco. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=sql-fine-tuning


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
output_dir = './sql_output_dir_premsql'
logging_dir = './sql_logging_dir_premsql'

In [6]:
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
clean_mem()

## Load and Prepare the Data

<|begin▁of▁sentence|> Question: What is the total sales amount? Context: The sales table contains information about each sale including date, amount, and customer ID. Answer: SELECT SUM(amount) FROM sales; <|end▁of▁sentence|>

In [8]:

from transformers import AutoTokenizer
from datasets import load_dataset

# Load a tokenizer to use its chat template
template_tokenizer = AutoTokenizer.from_pretrained(
    "premai-io/prem-1B-SQL"
)

# def format_prompt(sample):

#     """Given a sample dictionary with keys "title" and "abstract" format into a prompt.

#     Args:
#       sample: A sample dictionary from a Hugging Face dataset.

#     Returns:
#       sample: sample dictionary with "text" key for the formatted prompt.
#     """
#     #sample['text']=f"[INST] <> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <> {sample['question']} {sample['context']} [/INST] {sample['answer']}"
#     sample['text']=f"""<|begin▁of▁sentence|> You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.
# Context: {sample['context']}
# Question: {sample['question']}
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
#     return {"text":sample['text']}

def format_prompt(sample):
#     sample['text']=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
# You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>
# Context: {sample['context']}
# Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    sample['text'] = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {sample['context']}
Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    return {"text":sample['text']}
  

# Load and format the data using the template TinyLLama is using
dataset = (
    load_dataset("dpv/exoplanets-sql")#, split="train")
      .shuffle(seed=42)
)
dataset = dataset.map(format_prompt)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

In [10]:
dataset['train']['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: CREATE TABLE exoplanets (\n    name TEXT,\n    distance REAL,\n    stellar_magnitude REAL,\n    planet_type TEXT,\n    discovery_year INTEGER,\n    mass_multiplier REAL,\n    mass_wrt TEXT,\n    radius_multiplier REAL,\n    radius_wrt TEXT,\n    orbital_radius REAL,\n    orbital_period REAL,\n    eccentricity REAL,\n    detection_method TEXT\n); CREATE TABLE reference_planets (name TEXT, mass REAL);  'mass_wrt' in exoplanets table has a one-to-one match to 'name' in reference_planets table, and  'name' refers to either Earth or Jupyter, with Jupyter having 317.8 the mass of Earth.\nQuestion: What is the average mass of exoplanets discovered each year?<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:SELECT \n    e.discovery_y

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

- SFTTrainer expect a single 'text' column

## Evaluate the original model

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODEL_ID="premai-io/prem-1B-SQL"
model_orig = AutoModelForCausalLM.from_pretrained(MODEL_ID).to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.87it/s]


In [27]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
def compare_results(query_results, expected_results):
    """Compare the query results with the expected results."""
    # Sort the results for comparison
    query_results_sorted = sorted(query_results, key=lambda x: json.dumps(x, sort_keys=True))
    expected_results_sorted = sorted(expected_results, key=lambda x: json.dumps(x, sort_keys=True))
    return query_results_sorted == expected_results_sorted

valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model_orig.eval()
    with torch.no_grad():
        out = tokenizer.decode(model_orig.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+10:]
    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
generations = []
bad_idxs = []
result_comparisons = [False]*len(valid_loaded)
for idx,example in enumerate(valid_loaded):
    try:
        print('idx: ',idx)
        generation=generate_sql(context=example['context'], question=example['question'],debug=True)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
        bad_idxs.append(idx)
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        bad_idxs.append(idx)
    else:
        result_comparisons[idx]=compare_results(generation, example['results'])
    finally:
        generations.append(generation)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;

idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

 SELECT name, mass, discovery_year FROM (SELECT name, mass, discovery_year, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn FROM exoplanets) t WHERE rn = 1;  'rn' is the row number assigned by the ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn. If rn = 1, then the mass of the exoplanet is the minimum for that discovery year.

idx:  7


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year BETWEEN (SELECT MAX(discovery_year) - 2 FROM exoplanets) AND (SELECT MAX(discovery_year) FROM exoplanets);
idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius FROM exoplanets GROUP BY planet_type ORDER BY avg_orbital_radius DESC;
idx:  9
 SELECT name, (mass_wrt::REAL / radius_wrt::REAL) AS ratio FROM exoplanets ORDER BY ratio DESC LIMIT 5;  'mass_wrt' and 'radius_wrt' are the mass and radius of the exoplanet in the reference_planets table, respectively.

num_opp_errors: 3, num_other_exceptions:2,
Error rate: 50.0


In [25]:
op_errors, exceptions, result_comparisons

([(2, sqlite3.OperationalError('no such column: mass')),
  (6, sqlite3.OperationalError('no such column: mass')),
  (9, sqlite3.OperationalError('unrecognized token: ":"'))],
 [(3,
   sqlite3.ProgrammingError('You can only execute one statement at a time.')),
  (5,
   sqlite3.ProgrammingError('You can only execute one statement at a time.'))],
 [False, False, False, False, False, False, False, False, False, False])

In [26]:
generations

[' SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;\n',
 ' SELECT detection_method FROM exoplanets WHERE mass_wrt = (SELECT name FROM reference_planets WHERE mass = 317.8) GROUP BY detection_method ORDER BY COUNT(*) DESC LIMIT 1;\n',
 ' SELECT detection_method FROM exoplanets WHERE mass_wrt = (SELECT name FROM reference_planets WHERE mass = 317.8) GROUP BY detection_method ORDER BY COUNT(*) DESC LIMIT 1;\n',
 ' SELECT detection_method FROM exoplanets WHERE mass_wrt = (SELECT name FROM reference_planets WHERE mass = 317.8) GROUP BY detection_method ORDER BY COUNT(*) DESC LIMIT 1;\n',
 ' SELECT * FROM exoplanets WHERE discovery_year < 2000 AND eccentricity < 0.2;',
 ' SELECT * FROM exoplanets WHERE discovery_year < 2000 AND eccentricity < 0.2;',
 ' SELECT * FROM exoplanets WHERE discovery_year < 2000 AND eccentricity < 0.2;',
 ' SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year

In [28]:
bad_idxs

[2, 3, 5, 6, 9]

In [34]:
for idx in bad_idxs:
    print(run_query(generations[idx],cursor))
    print(valid_loaded[idx]['results'])
    print('_'*100)
#generations[2]

[]
[{"discovery_year": 2016, "total_mass": 2.3398372025261199e+30}, {"discovery_year": 2014, "total_mass": 1.92648023608052e+30}, {"discovery_year": 2022, "total_mass": 1.8723379260193998e+30}, {"discovery_year": 2021, "total_mass": 8.9332245596716e+29}, {"discovery_year": 2011, "total_mass": 6.552292609403601e+29}]
____________________________________________________________________________________________________
[]
[{"discovery_year": 1992, "name": "PSR B1257+12 c", "orbital_period": 0.18206708}, {"discovery_year": 1994, "name": "PSR B1257+12 b", "orbital_period": 0.06926762}, {"discovery_year": 1995, "name": "51 Pegasi b", "orbital_period": 0.011498973}]
____________________________________________________________________________________________________
[{"name": "47 Ursae Majoris b", "distance": 45.0, "stellar_magnitude": 5.03352, "planet_type": "Gas Giant", "discovery_year": 1996, "mass_multiplier": 2.53, "mass_wrt": "Jupiter", "radius_multiplier": 1.18, "radius_wrt": "Jupiter", 

In [38]:
#run_query("""SELECT name, mass, discovery_year FROM (SELECT name, mass, discovery_year, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn FROM exoplanets) t WHERE rn = 1;""",cursor)
run_query("""SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY orbital_period DESC) as rn FROM exoplanets) t WHERE rn <= 3; """,cursor)


'[{"name": "PSR B1257+12 d", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Super Earth", "discovery_year": 1992, "mass_multiplier": 3.9, "mass_wrt": "Earth", "radius_multiplier": 1.8, "radius_wrt": "Earth", "orbital_radius": 0.46, "orbital_period": 0.26885694, "eccentricity": 0.03, "detection_method": "Pulsar Timing", "rn": 1}, {"name": "PSR B1257+12 c", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Super Earth", "discovery_year": 1992, "mass_multiplier": 4.3, "mass_wrt": "Earth", "radius_multiplier": 1.91, "radius_wrt": "Earth", "orbital_radius": 0.36, "orbital_period": 0.18206708, "eccentricity": 0.02, "detection_method": "Pulsar Timing", "rn": 2}, {"name": "PSR B1257+12 b", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Terrestrial", "discovery_year": 1994, "mass_multiplier": 0.02, "mass_wrt": "Earth", "radius_multiplier": 0.338, "radius_wrt": "Earth", "orbital_radius": 0.19, "orbital_period": 0.06926762, "eccentricity": 0.0, "detect

## Configure Model, Data, Tokenizer, Trainer

In [12]:
clean_mem()

In [13]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_config,get_peft_model
MODEL_ID="premai-io/prem-1B-SQL" # just released Sept. 25, 2024, waiting for access!

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    #quantization_config = bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = "<pad>"
tokenizer.padding_side = "left"
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=128,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)


Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.49it/s]


In [14]:
from transformers import DataCollatorForLanguageModeling
# Step 3: Launch training with SFTTrainer
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs = 10.0,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
   # peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


 10%|█         | 10/100 [00:09<01:19,  1.14it/s]

{'loss': 0.9163, 'grad_norm': 2.0441858768463135, 'learning_rate': 0.00019510565162951537, 'epoch': 0.93}


 20%|██        | 20/100 [00:18<01:09,  1.15it/s]

{'loss': 0.2294, 'grad_norm': 1.3472613096237183, 'learning_rate': 0.00018090169943749476, 'epoch': 1.86}


...

100%|██████████| 100/100 [01:27<00:00,  1.15it/s]

{'loss': 0.0226, 'grad_norm': 0.2769683003425598, 'learning_rate': 1.973271571728441e-07, 'epoch': 9.3}


100%|██████████| 100/100 [01:40<00:00,  1.01s/it]

{'train_runtime': 101.0874, 'train_samples_per_second': 8.507, 'train_steps_per_second': 0.989, 'train_loss': 0.2133890298008919, 'epoch': 9.3}


TrainOutput(global_step=100, training_loss=0.2133890298008919, metrics={'train_runtime': 101.0874, 'train_samples_per_second': 8.507, 'train_steps_per_second': 0.989, 'total_flos': 1573369833062400.0, 'train_loss': 0.2133890298008919, 'epoch': 9.30232558139535})

In [15]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
def compare_results(query_results, expected_results):
    """Compare the query results with the expected results."""
    # Sort the results for comparison
    query_results_sorted = sorted(query_results, key=lambda x: json.dumps(x, sort_keys=True))
    expected_results_sorted = sorted(expected_results, key=lambda x: json.dumps(x, sort_keys=True))
    return query_results_sorted == expected_results_sorted

valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    #res = out[out.find('SQL Query:')+10:]
    res = out[out.find('SQL Query:')+10:out.find('<|end▁of▁sentence|>')]

    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
generations = []
bad_idxs = []
result_comparisons = [False]*len(valid_loaded)
for idx,example in enumerate(valid_loaded):
    try:
        print('idx: ',idx)
        generation=generate_sql(context=example['context'], question=example['question'],debug=True)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
        bad_idxs.append(idx)
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        bad_idxs.append(idx)
    else:
        result_comparisons[idx]=compare_results(generation, example['results'])
    finally:
        generations.append(generation)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


WITH YearlyDiscoveries AS (
    SELECT 
        discovery_year,
        COUNT(*) AS num_discoveries
    FROM exoplanets
    GROUP BY discovery_year
)
SELECT 
    discovery_year,
    num_discoveries
FROM YearlyDiscoveries
ORDER BY num_discoveries DESC
LIMIT 1; 
idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

WITH MinMassExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.discovery_year
    FROM exoplanets e
    WHERE e.mass_multiplier * r.mass = (SELECT MIN(mass_multiplier) FROM MassFactors)
)
SELECT 
    m.name,
    m.mass_multiplier * r.mass
FROM    exoplanets m
JOIN    reference_planets r ON m.mass_wrt = r.name
ORDER BY    m.discovery_year; 
idx:  7


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


SELECT 
    e.name,
    e.discovery_year,
    e.detection_method
FROM exoplanets e
WHERE e.discovery_year > 2000 
    AND e.detection_method != 'Earth'
ORDER BY e.discovery_year DESC; 
idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius
FROM exoplanets
GROUP BY planet_type
ORDER BY avg_orbital_radius DESC; 
idx:  9
WITH RankedExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.orbital_radius
    FROM exoplanets e
    ORDER BY e.mass_multiplier * r.mass DESC
    LIMIT 5
)

SELECT 
    e.name,
    e.mass_multiplier * r.mass AS mass,
    e.orbital_radius
FROM RankedExoplanets
WHERE mass_multiplier * r.mass = (SELECT mass FROM RankedExoplanets WHERE mass_rank = (SELECT mass_rank + 1 FROM RankedExoplanets))
ORDER BY mass_rank DESC; 
num_opp_errors: 2, num_other_exceptions:0,
Error rate: 20.0


In [17]:
op_errors, exceptions,result_comparisons

([(3, sqlite3.OperationalError('no such column: e.name')),
  (9, sqlite3.OperationalError('no such column: r.mass'))],
 [],
 [False, False, False, False, False, False, False, False, False, False])

- Not bad, brought error rate down from 50% to 20% correct query formatting within 10 epochs/2 minutes of training.  Results don't yield the same answers, though!  Could be lack of knowledge about the prompt structure for premsql, or perhaps the questions in the validation set need to be made clearer.